In [1]:
import os
os.getcwd()
os.chdir("H:\pyspark_advanced-coding_interview")
print(os.getcwd())


from pyspark.sql import SparkSession

# Create a Spark session with optimized settings
spark = (
    SparkSession.builder 
    .appName("OptimizedLocalSpark") 
    .config("spark.driver.memory", "8g")        
    .config("spark.executor.memory", "8g")    
    .config("spark.executor.cores", "4")       
    .config("spark.cores.max", "12")           
    .config("spark.sql.shuffle.partitions", "28")  
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") 
    .getOrCreate()
)
sc = spark.sparkContext

H:\pyspark_advanced-coding_interview


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count

# Initialize Spark session
spark = SparkSession.builder.appName("ConditionalCountExample").getOrCreate()

# Sample data
data = [
    (1, "John Doe", 101, 50000, "2020-01-15", 5),
    (2, "Jane Smith", 102, 60000, "2019-03-10", 4),
    (3, "Mike Brown", 101, 45000, "2021-06-25", 5),
    (4, "Emily Davis", 103, 70000, "2018-07-01", 3),
    (5, "James Wilson", 102, 55000, "2021-04-11", 4),
    (6, "Sara White", 101, 40000, "2022-09-19", 5)
]

# Create DataFrame
columns = ["EmployeeID", "FullName", "DeptID", "Salary", "HireDate", "ManagerID"]
df = spark.createDataFrame(data, columns)

# Show the DataFrame
df.show()

# Register the DataFrame as a temporary table for Spark SQL
df.createOrReplaceTempView("employee_table")


+----------+------------+------+------+----------+---------+
|EmployeeID|    FullName|DeptID|Salary|  HireDate|ManagerID|
+----------+------------+------+------+----------+---------+
|         1|    John Doe|   101| 50000|2020-01-15|        5|
|         2|  Jane Smith|   102| 60000|2019-03-10|        4|
|         3|  Mike Brown|   101| 45000|2021-06-25|        5|
|         4| Emily Davis|   103| 70000|2018-07-01|        3|
|         5|James Wilson|   102| 55000|2021-04-11|        4|
|         6|  Sara White|   101| 40000|2022-09-19|        5|
+----------+------------+------+------+----------+---------+



----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 53703)
Traceback (most recent call last):
  File "c:\Users\lpdda\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "c:\Users\lpdda\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "c:\Users\lpdda\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "c:\Users\lpdda\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 755, in __init__
    self.handle()
  File "C:\spark\python\pyspark\accumulators.py", line 281, in handle
    poll(accum_updates)
  File "C:\spark\python\pyspark\accumulators.py", line 253, in poll
    if func():
       ^^^^^^
  File "C:\spark\python\py

# when function along with count to perform a conditional count.

In [2]:
# Conditional count: count the number of employees in DeptID 101
df_conditional_count = df.withColumn("Dept101Count", when(col("DeptID") == 101, 1).otherwise(0))

# Count the number of rows where the condition is true
count_df = df_conditional_count.groupBy().sum("Dept101Count").withColumnRenamed("sum(Dept101Count)", "CountDept101")
count_df.show()


+------------+
|CountDept101|
+------------+
|           3|
+------------+



In [3]:
# SQL query for conditional count
result_sql = spark.sql("""
    SELECT COUNT(*) AS CountDept101
    FROM employee_table
    WHERE DeptID = 101
""")
result_sql.show()


+------------+
|CountDept101|
+------------+
|           3|
+------------+



In [4]:
# Conditional count using filter
count_filtered = df.filter(col("DeptID") == 101).count()
print(f"Number of employees in DeptID 101: {count_filtered}")


Number of employees in DeptID 101: 3


In [6]:
res = spark.sql("""
                
    SELECT DeptID,COUNT(*) AS EmpCnt, 
    Sum( case when salary between 30000 and 80000 then 1 else 0 end ) as EmpCnt1
    FROM employee_table
    Group by DeptID
   
""")
res.show()

+------+------+-------+
|DeptID|EmpCnt|EmpCnt1|
+------+------+-------+
|   101|     3|      3|
|   102|     2|      2|
|   103|     1|      1|
+------+------+-------+

